# Punto 3

In [4]:
//Importar librerías
%use dataframe

In [33]:
import javax.xml.crypto.Data
import java.util.*
import kotlin.collections.*
var df_terms = DataFrame.readCSV("./terms.csv") // Cambia la ruta al archivo CSV
val df_resources = DataFrame.readCSV("../../resources/collection.csv") // Cambia la ruta al archivo CSV

# Logica

In [35]:

fun sortAndCleanAbstract(abstract: String): kotlin.collections.List<String> = abstract
    .lowercase(Locale.getDefault())
    .split(" ")
    .asSequence()
    .map { it.trim() }
    .map { if (it.isNotEmpty() && it.last() == '.') it.dropLast(1) else it }
    .filter { it.isNotEmpty() }
    .sorted()
    .toList()


fun countWordOccurrences(word: String, sortedList: kotlin.collections.List<String>): Int {
    // Find any occurrence with binary search
    val index = Collections.binarySearch(sortedList, word)

    // Word not found
    if (index < 0) {
        return 0
    }

    // Count all occurrences (left and right of found index)
    var count = 1

    // Check left side
    var left = index - 1
    while (left >= 0 && sortedList[left] == word) {
        count++
        left--
    }

    // Check right side
    var right = index + 1
    while (right < sortedList.size && sortedList[right] == word) {
        count++
        right++
    }

    return count
}

fun countSetOfTerms(setOfTerms: kotlin.collections.Set<String>, sortedList: kotlin.collections.List<String>): Int {
    var count = 0
    for (term in setOfTerms) {
        count += countWordOccurrences(term, sortedList)
    }
    return count
}



In [17]:
// Imprimir el esquema de la base de datos
.schema()

Categoria: String
Variable: String
Sinonimos: String?

In [19]:
import java.util.*

// Convert both "Variable" and "Sinonimos" columns to lowercase
df_terms = df_terms
.update { Variable }
.with { it.lowercase(Locale.getDefault()) }

.update { Sinonimos }
.with { it?.lowercase(Locale.getDefault()) }


df_terms

Categoria,Variable,Sinonimos
Habilidades,abstraction,null
Habilidades,algorithm,null
Habilidades,algorithmic thinking,null
Habilidades,coding,null
Habilidades,collaboration,null
Habilidades,cooperation,null
Habilidades,creativity,null
Habilidades,critical thinking,null
Habilidades,debug,null
Habilidades,decomposition,null


# Frecuencia de aparición de términos en la categoría "Habilidades"

El siguiente bloque de codigo une los resultados de la columna "variable" y la columna "sinonimos" en una sola lista y la almacena en una variable (tambien una lista de listas)
 llamada "terms".

Es decir, tenemos una lista de listas con los terminos y sus respectivos sinonimos.

In [32]:
// Create a list of lists where each inner list contains the variable and its synonyms
// El siguient bloque de codigo
val terms = mutableListOf<kotlin.collections.List<String>>()

// Process each row of the dataframe
df_terms.rows().forEach { row ->
    val variable = row["Variable"] as String
    val synonymsStr = row["Sinonimos"] as String?

    // Create a mutable list starting with the variable
    val termList = mutableListOf(variable)

    // Add synonyms if they exist
    if (!synonymsStr.isNullOrEmpty()) {
        // Split synonyms by "|" and add them to the list
        termList.addAll(synonymsStr.split("|").map { it.trim() })
    }

    // Add the complete list to our collection
    terms.add(termList.toList())
}

// Display the first 5 entries to verify
terms.filter { it.size > 1 }.forEach { println(it) }

[classical test theory, ctt]
[confirmatory factor analysis, cfa]
[exploratory factor analysis, efa]
[item response theory, irt]
[structural equation model, sem]
[beginners computational thinking test, bctt]
[coding attitudes survey, escas]
[competent computational thinking test, cctt]
[computational thinking skills test, ctst]
[computational thinking assessment for chinese elementary students, cta-ces]
[computational thinking challenge, ctc]
[computational thinking levels scale, ctls]
[computational thinking scale, cts]
[computational thinking skill levels scale, cts]
[computational thinking test, ctt]
[computational thinking test for lower primary, cttlp]
[computerized adaptive programming concepts test, capct]
[ct scale, cts]
[elementary student coding attitudes survey, escas]
[upper elementary education, upper elementary school]
[primary school, primary education, elementary school]
[early childhood education, kindergarten, preschool]
[secondary school, secondary education]
[high sc

In [39]:
//Obtenemos la lista de abstracts.
val list_of_abstracts: List<String> = df_resources.get { Abstract }.filter { !it.isNullOrEmpty() }.toList() as List<String>
list_of_abstracts.take(10)

[The digital paradigm requires efficient methods of teaching CAAD tools in architecture schools. With the trend of enhancing the design process with parametric methods, linking architecture with other knowledge areas, such as mathematics, is gaining in importance. Equipping future architects with skills in algorithmic thinking is yet another challenge for education. This paper describes the workflow of an early-stage course addressing this challenge, conducted at the Warsaw University of Technology’s Faculty of Architecture. The course focuses on the students’ ability to construct complex geometric forms in the digital environment by introducing an extensive analytic phase. The students study the geometric foundations of real-world architectural cases and translate them into parametric models. Later, they explore the potential of the generated solutions space. The results compare the course’s teaching efficiency with the outcomes of past courses covering similar subjects. , Computation

In [41]:
val frequency_of_terms : List<Pair<String, Int>> = terms.map { term ->
    val termSet = term.toSet()
    val count = countSetOfTerms(termSet, sortAndCleanAbstract(list_of_abstracts[0]))
    Pair(term.joinToString(", "), count)
}

frequency_of_terms
    .sortedByDescending { it.second }
    .take(10)
    .forEach { println(it) }

[abstraction]


0